# Recomendaciones proyecto


1. **Rendimiento esperado:**  
   - No es necesario alcanzar un rendimiento cercano al **100%** para obtener la nota máxima en el proyecto. Se evaluará la calidad del enfoque, la justificación de las decisiones tomadas y la interpretación de los resultados obtenidos.  

2. **Uso de muestras en caso de limitaciones:**  
   - Si su equipo no puede cargar o procesar todos los datos debido a limitaciones de recursos (como RAM), puede trabajar con **muestras representativas del conjunto de datos**.  
   - Es **obligatorio justificar** este enfoque en el informe, explicando claramente las razones detrás de la decisión y cómo se asegura que las muestras son representativas.  

3. **Optimización de recursos:**  
   - Si enfrenta problemas de memoria al ejecutar tareas, **reduzca la cantidad de procesos paralelos** (**jobs**) a un nivel que su computador o intérprete web pueda manejar. Es preferible priorizar la estabilidad del proceso sobre la velocidad.

4. **Paralelización para búsquedas de hiperparámetros:**  
   - Aproveche la paralelización para acelerar la búsqueda de hiperparámetros, especialmente si esta es un cuello de botella en su proyecto. Herramientas como `GridSearchCV`, `RandomizedSearchCV` o `Optuna` suelen permitir paralelización configurando el parámetro `n_jobs`.  

5. **Grillas de búsqueda razonables:**  
   - Al realizar búsquedas de hiperparámetros, **diseñe grillas de búsqueda razonables** que no sean excesivamente grandes.  
   - Recuerde que, aunque explorar un mayor espacio de hiperparámetros puede parecer atractivo, también puede hacer que el proceso sea extremadamente lento o inviable. Ajuste el tamaño de las grillas para garantizar que la búsqueda **converja en tiempos razonables** y no tome **"3.5 eternidades"**.



A continuación, se detallan las tareas que deben completar para la **Entrega Parcial 1**. Recuerden que el principal entregable es **predecir, con su mejor modelo, sobre los clientes del archivo `X_t1.parquet` y subir las predicciones a la plataforma CodaLab**. **Para esta entrega el desarrollo del informe es sugerido, pero no mandatorio**. Asegúrense de utilizar el archivo correspondiente para subir los resultados a la competencia. El uso de datos incorrectos se reflejará en un bajo desempeño en la tabla de clasificación.

### **Tareas a realizar:**
1. **Análisis exploratorio de datos:**  
   Realicen un análisis detallado para identificar patrones, tendencias y relaciones en los datos. Este paso les permitirá comprender mejor las características del conjunto de datos y guiar las siguientes decisiones en el pipeline de modelamiento.

2. **Preprocesamiento de datos:**  
   Incluyan técnicas de preprocesamiento que aseguren la calidad y adecuación de los datos para los modelos. Algunas tareas sugeridas son:  
   - Estandarización de filas y/o columnas.  
   - Reducción de dimensionalidad.  
   - Discretización de variables numéricas a categóricas.  
   - Manejo de datos nulos.  
   - Otras transformaciones relevantes según los datos disponibles.  

3. **División del conjunto de datos:**  
   Implementen una técnica **Hold-Out**, separando el conjunto de datos en **70% para entrenamiento** y **30% para testeo**.

4. **Creación de un modelo baseline:**  
   Entrenen un modelo sencillo que sirva como línea base para comparar el rendimiento de los modelos más avanzados.

5. **Desarrollo de 3 modelos de Machine Learning diferentes al baseline:**  
   - Utilicen exclusivamente pipelines de **Scikit-Learn** para esta iteración del proyecto.  
   - El uso de librerías o herramientas externas será penalizado con una calificación de **0** en las secciones implicadas.  

6. **Interpretabilidad del modelo con mejores resultados:**  
   Apliquen técnicas que permitan interpretar y justificar los resultados obtenidos por el modelo con mejor desempeño.

----------------------------------------------

# Informe Proyecto Lab
Integrantes: 
- Alonso Uribe
- Carolina Núñez
  
Grupo: Adaval


#### **1. Introducción [0.25 puntos]**  
Esta sección debe incluir:  
- Una breve descripción del problema planteado: ¿Qué se intenta predecir?  
- Un resumen de los datos de entrada proporcionados.  
- La métrica seleccionada para evaluar los modelos, justificando su elección. Considerando que los datos están desbalanceados, eviten usar `accuracy` y enfoquen su análisis en métricas como `precision`, `recall` o `f1-score`, indicando en qué clase se centrarán.  
- Una mención breve de los modelos utilizados para resolver el problema, incluyendo las transformaciones intermedias aplicadas a los datos.  
- Un cierre con un análisis general de los resultados obtenidos, indicando si el modelo final cumplió con los objetivos y cómo se posicionaron respecto a otros equipos.

In [ ]:
import datetime
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import defaultdict
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer


maxcols = pd.get_option("display.max_columns")
pd.set_option("display.max_columns", None)

In [ ]:
# para entrenar
X_t0 = pd.read_parquet('X_t0.parquet')
y_t0 = pd.read_parquet('y_t0.parquet')
# para predecir(?)
X_t1 = pd.read_parquet('X_t1.parquet')
X_t0 = X_t0[sorted(X_t0.columns)]
X_t1 = X_t1[sorted(X_t1.columns)]

In [ ]:
pd.set_option("display.max_columns", maxcols )
X_t0.info()

In [ ]:
# Verificamos que no hayan valores nulos
X_t0.isnull().sum()

In [ ]:
numeric_var = X_t0.select_dtypes(include=['number']).columns
object_var = X_t0.select_dtypes(include=['object']).columns
timestamps = pd.Index([col for col in X_t0.columns if "timestamp" in col])
numeric_var = numeric_var.drop(timestamps)
timestamps = timestamps.drop('risky_first_last_tx_timestamp_diff')
ts_diff_col = pd.Index(['risky_first_last_tx_timestamp_diff'])

In [ ]:
X_t0[timestamps].map(lambda ms: datetime.datetime.fromtimestamp(ms))

Observamos que las columnas timestamp hablan de fechas de transacción. Sin conocimiento del negoción, hablan de una fecha inicio y termino del prestamo(?)

In [ ]:
(X_t0[ts_diff_col[0]] == (X_t0['risky_last_tx_timestamp']) - X_t0['risky_first_tx_timestamp']).sum()/ len(X_t0)

Aquí podemos ver que la columna `risky_first_last_tx_timestamp_diff` o corresponde a la diferencia entre las columnas `risky_last_tx_timestamp` y `risky_first_tx_timestamp`.  
Por lo tanto debemos decidir que necesitamos del fenómeno, los valores absolutos en el tiempo, o solo la diferencia de estos valores. Concluimos que estos tres valores junto no entraran al modelo.

In [ ]:
# Busca las variables binarias a partir del número de valores distintos que tenga
weird = defaultdict(list)
threshold = 0.5
for col in X_t0[numeric_var]:
    diff_values = len(X_t0[col].value_counts())
    min_prop = (X_t0[col] == X_t0[col].min()).sum() / len(X_t0[col]) 
    max_prop = (X_t0[col] == X_t0[col].max()).sum() / len(X_t0[col])
    is_binary = diff_values == 2
    is_weird = False
    if min_prop > threshold and not is_binary :
        weird['min_prop'].append(col)
        is_weird = True
    if max_prop > threshold and not is_binary :
        weird['max_prop'].append(col)
        is_weird = True
    if is_binary:
        weird['binary'].append(col)
        is_weird = True
    if not is_weird:
        weird['not'].append(col)
    print(f"{col}:\n # valores distintos: {diff_values}\n Proporción de valor mínimo: {min_prop}\n Proporción de valor máximo: {max_prop};\n\n")

In [ ]:
for name, keys in weird.items():
    print(f"{name.upper()}:\n {', '.join(sorted(keys))}\n")

In [ ]:
pd.set_option("display.max_columns", None)
X_t0[weird['max_prop']].describe()

In [ ]:
X_t0[weird['min_prop']].describe()


### Test de Anderson

Para determinar que variables se comportan de manera gaussiana, se implementará el test de D'Agostino and Pearson con nivel de significancia del 95%, cuya hipótesis nula es que los datos provienen de una distribución dada. En este caso, de una distribución normal. Entonces, si el p-value es menor al nivel de significancia se rechaza la nula, es decir, los datos no provienen de una distribución normal.

(Este test es similar al Test de Kolmogorov-Smirnov, sin embargo funciona bien para muestras grandes)

In [ ]:
anderson(np.random.normal(0, 1, len(X_t0['market_cmo'])), dist='norm')

In [ ]:
kstest(data[numeric_var[1]], 'norm')

In [ ]:
# D'Agostino and Pearson test
from scipy.stats import kstest, norm, anderson, normaltest

# Test de normalidad
data = X_t0[numeric_var].copy()

normal_var = []
for var in numeric_var:
    statistic, p_value = normaltest(data[var])
    # print(f'Variable: {var}\n statistic: {statistic}    p_value: {p_value}\n')
    if p_value >= 0.05 : # No se rechaza la nula -> Los datos distribuyen normal
        normal_var.append(var)

normal_var

Para sorpresa nuestra, según los test realizados, ninguna de las variables numéricas distribuye normal.

In [ ]:
from numpy import random

# plt.hist(X_t0['market_apo'], color='blue', label='market_apo');
plt.hist((X_t0['market_apo']-X_t0['market_apo'].mean())/X_t0['market_apo'].std(), color='blue', label='market_apo');
plt.hist(np.random.normal(0, 1, len(X_t0['market_cmo'])), alpha=0.5, color='red', label='normal');
plt.legend()

Observamos que estas variables tienen datos atípicos para una variable numérica

In [ ]:
numeric_var = numeric_var.drop(weird['binary'])
to_datetime_func = FunctionTransformer(lambda df: df.map(lambda ms: datetime.datetime.fromtimestamp(ms)))

first_transformer = ColumnTransformer([
    ('numeric', StandardScaler(), numeric_var),
    ('binary', MinMaxScaler(), weird['binary']),
    ('ts', to_datetime_func, timestamps), # ms a Timestamp según fecha POSIX
    ('tsdiff', FunctionTransformer(lambda ms: ms/1000), ts_diff_col) # ms a segundos
    ], remainder='passthrough')
    
first_transformer.set_output(transform='pandas')
optimus = first_transformer.fit_transform(X_t0)
optimus.columns = [col.split('_', 1)[-1].lstrip('_') for col in optimus.columns]

In [ ]:
optimus

In [ ]:
vars = object_var.insert(len(object_var), 'risky_first_tx_timestamp')
for _ ,df in optimus[vars].groupby(*object_var):
    print(df['risky_first_tx_timestamp'].value_counts())

In [ ]:
import numpy as np 
import scipy.stats as stats
import matplotlib.pyplot as plt
def plot_qqplot(df, columns, num_cols=7, dist='norm'):
    columns = columns if not isinstance(columns, str) else [columns]
    num_rows = int(len(columns)/num_cols)+min(1, len(columns)%num_cols)
    fig, axes = plt.subplots(num_rows, num_cols if num_rows != 1 else len(columns), sharex=True)
    axes = axes.reshape(-1) if isinstance(axes, type(np.array([]))) else [axes]
    fig.set_figwidth(26 if num_rows > 1 else 3*len(columns))
    fig.set_figheight(num_rows*3)
    for i, col in enumerate(columns):
        data, _ = stats.probplot(df[col], dist=dist, plot=axes[i])
        axes[i].set_title(col)
        if i!=0:
            axes[i].set_ylabel('')
        if isinstance(axes, type(np.array([]))) and i!=(len(axes)-1):
            axes[i].set_xlabel('')
    fig.set
    plt.show()

In [ ]:
plot_qqplot(optimus, numeric_var)

In [ ]:
plot_qqplot(optimus, ts_diff_col)

In [ ]:
plot_qqplot(optimus, weird['binary'])

qqplot esperado para varible binaria

In [ ]:
plot_qqplot(optimus, weird['max_prop'])

Este campo se asumirá como variable binaria

In [ ]:
print(", ".join(weird['min_prop']))
plot_qqplot(optimus, weird['min_prop'])

Podemos observar en las primeras 5 variables un comportamiento parecido a una variable binaria. Para facilitar la predicción al modelo, podemos definir para ellas 3 o 4 bins, donde los bins 2 y 3 serían valores de transición desde los menores a los mayores valores encontrados.

Para las demás variables, vemos comportamientos interesantes. Por ejemplo para `max_market_drawdown_365d` se observan valores escalonados, lo que también podría permitirnos simplificar estos valores en bins. A su vez, vemos que varias se comportan de manera normal.

In [ ]:
pd.set_option("display.max_columns", maxcols)
# Histogramas para revisar el comportamiento de las variable
axes = X_t0[X_t0.drop(columns=weird['not']).columns].hist(figsize=(16,12), bins=15)
for ax in axes.flatten():
    ax.set_title(ax.get_title(), fontsize=10)
    ax.tick_params(axis='x', labelsize=6)
    ax.tick_params(axis='y', labelsize=6)

In [ ]:
axes = X_t0[weird['not']].hist(figsize=(20,15), bins=15)
for ax in axes.flatten():
    ax.set_title(ax.get_title(), fontsize=10)
    ax.tick_params(axis='x', labelsize=6)
    ax.tick_params(axis='y', labelsize=6)

In [ ]:
X_corr = X_t0[weird['not']].corr().stack()
X_corr = X_corr[X_corr.index.get_level_values(0) != X_corr.index.get_level_values(1)]
X_corr

In [ ]:
X_corr = X_t0[weird['not']].corr()

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = X_corr.columns,
        y = X_corr.index,
        z = np.array(X_corr),
        text=X_corr.values,
        texttemplate='%{text:.2f}'
    )
)
fig.update_layout(title_text='Correlación entre las variables numéricas',
                    height=800, width=1400)
fig.show()

#### **2. Modelos con Scikit-Learn (Entrega Parcial 1)**

##### **2.1. Análisis Exploratorio de Datos [0.5 puntos]**  
Realicen un análisis que explore patrones, tendencias y relaciones clave en los datos. Incluyan:  
- Estadísticas descriptivas generales.  
- Visualizaciones para identificar distribuciones, valores atípicos y posibles relaciones entre variables.  
- Cualquier observación relevante que pueda influir en las etapas posteriores del proyecto.  

El informe debe ser detallado y profesional, demostrando no solo la implementación técnica, sino también una comprensión profunda de los datos y la problemática planteada.



##### **2.2 Preprocesamiento de Datos [0.25 puntos]**

Esta sección se centra en la limpieza y preparación de los datos para garantizar que sean adecuados para el entrenamiento y evaluación de los modelos. Es fundamental ejecutar un **`train_test_split`** para dividir los datos en conjuntos de entrenamiento y validación, siguiendo la proporción establecida (por ejemplo, 70/30). 

Se espera la implementación de diversas técnicas de preprocesamiento, tales como:  
- **Uso de `ColumnTransformer`:** Permite aplicar transformaciones específicas a diferentes columnas de manera eficiente.  
- **Imputación de valores nulos:** Elija una estrategia adecuada (media, mediana, moda, etc.) para completar los datos faltantes.  
- **Discretización de variables:** Convierte variables numéricas continuas en categóricas, si resulta útil para el modelo.  
- **Estandarización o normalización:** Mejora el rendimiento de algunos algoritmos que son sensibles a la escala de los datos.  
- Otras transformaciones necesarias dependiendo de las características específicas del conjunto de datos.

El proceso debe estar bien documentado y justificado en el informe, explicando las decisiones tomadas en función de los datos y los objetivos del proyecto.

---

In [ ]:
y_t0.value_counts()

Dado que las clases están en proporciones relativamente similares, no consideramos necesario 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(optimus, y_t0.values, 
                                                    test_size=0.3, random_state=29, shuffle=True,)

##### **2.3 Baseline [0.25 puntos]**

En esta sección se debe construir el modelo más sencillo posible que pueda resolver el problema planteado, conocido como **modelo baseline**. Su propósito es servir como referencia para comparar el rendimiento de los modelos más avanzados desarrollados en etapas posteriores.  

Pasos requeridos:  
- Implemente, entrene y evalúe un modelo básico utilizando un pipeline.  
- Asegúrese de incluir en el pipeline las transformaciones del preprocesamiento realizadas previamente junto con un clasificador básico.  
- Evalúe el modelo y presente el informe de métricas utilizando **`classification_report`**.  

Documente claramente cómo se creó el modelo, las decisiones tomadas y los resultados obtenidos. Este modelo será la base comparativa en las secciones posteriores.

---

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Aplico labelEncoder a la col 'wallet_address'

encoder = LabelEncoder()
encoder.fit(X_t0.wallet_address)

X_train.wallet_address  = encoder.transform(X_train.wallet_address)
X_test.wallet_address = encoder.transform(X_test.wallet_address)

In [ ]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(X_train, y_train)
y_dummy = dummy_clf.predict(X_test)

# dummy.score:Return the mean accuracy on the given test data and labels.
dummy_clf.score(X_test, y_test)

In [ ]:
lsvc = LinearSVC(random_state=29)
lsvc.fit(X_train.drop(columns=timestamps), y_train)
y_lsvc = lsvc.predict(X_test.drop(columns=timestamps))
lsvc.score(X_test.drop(columns=timestamps), y_test)

In [ ]:
# AUC curve
models = ['Dummy clf', 'LinearSVC']

for i, y in enumerate([y_dummy, y_lsvc]):
    fpr, tpr, thresholds = roc_curve(y_test, y, pos_label=2)
    auc(fpr, tpr)
    fpr, tpr, _ = roc_curve(y_test, y,)
    roc_auc = roc_auc_score(y_test, y)

    plt.plot(fpr, tpr, lw=2, label=f'Curva ROC {models[i]}(area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.3)
plt.legend()

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')

In [ ]:
print(classification_report(y_test, y_dummy))
print(classification_report(y_test, y_lsvc))

In [ ]:
lsvc.score(X_test.drop(columns=timestamps), y_test)

The precision-recall curve shows the tradeoff between precision and recall for different thresholds. A high area under the curve represents both high recall and high precision. High precision is achieved by having few false positives in the returned results, and high recall is achieved by having few false negatives in the relevant results. High scores for both show that the classifier is returning accurate results (high precision), as well as returning a majority of all relevant results (high recall).

In [ ]:
y_score = lsvc.decision_function(X_test.drop(columns=timestamps))

display = PrecisionRecallDisplay.from_predictions(
    y_lsvc, y_score, name="LinearSVC", plot_chance_level=True
)
_ = display.ax_.set_title("2-class Precision-Recall curve")